# Transfer Learning using Keras and ResNet-50 by IS - First Try
* 데이터셋은 64종류의 과일로 이루어진 42,345장의 이미지로 이루어져 있다. 
* 일반적인 접근방법과 전이 학습으로의 접근방법을 비교해볼 것이다.
* 2 epoch로 98.44%의 정확도를 얻을 수 있다.

**목차**는 다음과 같다:
1. 전이 학습(Transfer Learning)에 대한 간단한 설명
2. Kaggle Kernel을 이용한 전이 학습
3. 데이터를 로드하고 시각호
4. 모델 설계 및 컴파일
5. 미리 학습된 모델을 학습시키고 검증
6. 바닐라 모델(내가 순수 만든 모델)을 학습시키고 검증
7. 미리 학습된 모델과 바닐라모델과의 비교

In [122]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

In [123]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
import tensorflow as tf

## 1. Transfer Learning
전이 학습에서 우리는 먼저 기본 데이터와 작업에 기반을 한 기본 신경망을 학습시키고 학습된 피처의 목적을 변경하거나 대상 데이터셋과 작업에 대한 학습을 할 두 번째 대상 신경망으로 전달한다. 이 과정은 피처가 기본 작업에 특정되지 않고 기본 작업과 대상 작업 모두에 적합한 일반적인 피처라면 잘 작동하는 경향이 있다.

Lisa Torrey와 Jude Shavlik은 전이 학습에 대한 챕터에서 전이학습을 사용할 때의 3가지 이점을 설명하고 있다:
* 더 빠른 출발이 가능하다 : 소스 모델의 초기 스킬이(모델을 정제하기 전) 그렇지 않은 경우보다 높다.
* 더 높은 경사 : 소스 모델을 학습시키는 동안 스킬의 향상속도가 그렇지 않은 경우보다 더 깊다.
* 더 높은 점근선(asymptote) : 학습된 모델의 수렴 스킬이 그렇지 않은 모델보다 더 좋다.

![transfer-learning-benefits](../img/transfer-learning-benefits.png)

기본적으로 미리 학습된 모델(다른 누군가에 의해 매우 큰 데이터셋으로 학습이 완료된 신경망의 가중치와 파라미터들)을 가져와서 우리 자신의 데이터셋으로 "세밀 조정(fine tuning)"을 한다. 전이 학습의 아이디어는 미리 학습된 모델은 초기화된 가중치를 제공하여 수렴 속도를 높이거나 관심 업무를 위한 고정 피처추출기 역할을 하는 것이다.

두 가지 주요 전이학습 시나리오는 다음과 같다:
* ConvNet 세밀 조정(Fine Tuning)하기: 임의의 값으로 가중치를 초기화시키는 대신 미리 학습된 신경망의 가중치로 신경망을 초기화한다. 예를 들면 ImageNet 1000과 같은 큰 데이터셋으로 미리 훈련된 신경망 말이다. 이 시나리오에서는 전체적인 신경망이 우리과 관심있는 데이터셋으로 다시 학습되어야 할 필요가 있다.
* ConvNet을 고정 피처 추출기로 사용하기 : 여기서 최종 완결연결계층을 제외한 모든 신경망에 대한 가중치를 동결(freeze)한다. (멈춘다의 의미인가?) 이 마지막 완전연결 계층은 임의의 가중치로 이루어진 새로운 층으로 대체되고 이 층만 학습시킨다.

이 노트북에서는 첫 번째 시나리오를 따른다.

## 2. Transfer Learning using Kaggle Kernels
### 2.1. Using the Keras Pretrained Models dataset
캐글 커널은 네트워크 연결이 되지 않으므로 미리 훈련된 케라스 모델을 다운로드 받을 수 없다. (주피터에서는 가능) [이 데이터셋](https://www.kaggle.com/moltean/fruits)은 우리가 원하는 미리 학습된 모델을 캐글 커널 환경에서 사용할 수 있도록 도와준다.

우리가 해줘야 할 것은 케라스가 찾고 있는 캐시 디렉토리(~/.keras/models)에 미리 학습된 모델을 복사하는 것이다.

In [124]:
# 케라스에서만
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [125]:
# 아래 모델들은 Github에서 공유되고 있는 imagenet1000의 모델을 가져오는 듯
!copy ../data/keras-pretrained-models/*notop* ~/.keras/models/
!copy ../data/keras-pretrained-models/imagenet_class_index.json ~/.keras/models/
!copy ../data/keras-pretrained-models/resnet50* ~/.keras/models/

print("Available Pretrained Models:\n")
!dir ~/.keras/models

명령 구문이 올바르지 않습니다.
명령 구문이 올바르지 않습니다.
명령 구문이 올바르지 않습니다.
Available Pretrained Models:



스위치가 틀립니다 - ".keras".


## 3. Reading and Visualizing the Data
### 3.1. Reading the Data
케라스의 나머지 부분과 마찬가지로 이미지 Augmentation API가 간단하고 엄청나다. 우리는 ImageDataGenerator를 사용하여 데이터를 가져와 신경망에 넣을 것이다.

케라스는 ImageDataGenerator 클래스를 제공하는데 이 클래스는 이미지 데이터 준미 및 Augmentation에 대한 설정을 정의한다. 메모리 내 전체 이미지 데이터셋에 대한 작업을 수행하는 대신, 이 API는 딥러닝 모델 학습 과정에 의해 반복되도록(iterated) 설계되어 딱딱 Augmented 이미지를 생성한다. 이렇게 하면 메모리 부하를 줄일 수 있지만, 모델 학습 중에 약간의 시간 비용이 추가된다.

데이터 생성기 자체는 요청 시 디렉토리에서 이미지 샘플 배치를 반환하는 사실상 이터레이터(iterator)이다. `flow_from_directory()` 함수를 호출하여 배치 크기를 정하고 데이터 생성기를 준비하면 이미지의 배치를 얻을 수 있다.

In [126]:
# 이미지의 차원
# 미리 학습된 모델의 이미지 크기에 맞춤
img_width, img_height = 224, 224

train_data_dir = '../data/Training'
validation_data_dir = '../data/Test'
nb_train_samples = 31688
nb_validation_samples = 10657
batch_size=16

In [127]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [128]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


### 3.2. Visualizing the Data

In [129]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [130]:
training_data = pd.DataFrame(train_generator.classes, columns=['classes'])
testing_data = pd.DataFrame(validation_generator.classes, columns=['classes'])

In [131]:
def create_stack_bar_data(col, df):
    aggregated = df[col].value_counts().sort_index()
    x_values = aggregated.index.tolist()
    y_values = aggregated.values.tolist()
    return x_values, y_values

In [132]:
x1, y1 = create_stack_bar_data('classes', training_data)
x1 = list(train_generator.class_indices.keys())
x1, y1

(['Apple Braeburn',
  'Apple Crimson Snow',
  'Apple Golden 1',
  'Apple Golden 2',
  'Apple Golden 3',
  'Apple Granny Smith',
  'Apple Pink Lady',
  'Apple Red 1',
  'Apple Red 2',
  'Apple Red 3',
  'Apple Red Delicious',
  'Apple Red Yellow 1',
  'Apple Red Yellow 2',
  'Apricot',
  'Avocado',
  'Avocado ripe',
  'Banana',
  'Banana Lady Finger',
  'Banana Red',
  'Beetroot',
  'Blueberry',
  'Cactus fruit',
  'Cantaloupe 1',
  'Cantaloupe 2',
  'Carambula',
  'Cauliflower',
  'Cherry 1',
  'Cherry 2',
  'Cherry Rainier',
  'Cherry Wax Black',
  'Cherry Wax Red',
  'Cherry Wax Yellow',
  'Chestnut',
  'Clementine',
  'Cocos',
  'Corn',
  'Corn Husk',
  'Cucumber Ripe',
  'Cucumber Ripe 2',
  'Dates',
  'Eggplant',
  'Fig',
  'Ginger Root',
  'Granadilla',
  'Grape Blue',
  'Grape Pink',
  'Grape White',
  'Grape White 2',
  'Grape White 3',
  'Grape White 4',
  'Grapefruit Pink',
  'Grapefruit White',
  'Guava',
  'Hazelnut',
  'Huckleberry',
  'Kaki',
  'Kiwi',
  'Kohlrabi',
  'Ku

In [133]:
trace1 = go.Bar(x=x1, y=y1, opacity=0.75, name='class count')
trace1

Bar({
    'name': 'class count',
    'opacity': 0.75,
    'x': [Apple Braeburn, Apple Crimson Snow, Apple Golden 1, Apple Golden 2,
          Apple Golden 3, Apple Granny Smith, Apple Pink Lady, Apple Red 1, Apple
          Red 2, Apple Red 3, Apple Red Delicious, Apple Red Yellow 1, Apple Red
          Yellow 2, Apricot, Avocado, Avocado ripe, Banana, Banana Lady Finger,
          Banana Red, Beetroot, Blueberry, Cactus fruit, Cantaloupe 1, Cantaloupe
          2, Carambula, Cauliflower, Cherry 1, Cherry 2, Cherry Rainier, Cherry Wax
          Black, Cherry Wax Red, Cherry Wax Yellow, Chestnut, Clementine, Cocos,
          Corn, Corn Husk, Cucumber Ripe, Cucumber Ripe 2, Dates, Eggplant, Fig,
          Ginger Root, Granadilla, Grape Blue, Grape Pink, Grape White, Grape White
          2, Grape White 3, Grape White 4, Grapefruit Pink, Grapefruit White,
          Guava, Hazelnut, Huckleberry, Kaki, Kiwi, Kohlrabi, Kumquats, Lemon,
          Lemon Meyer, Limes, Lychee, Mandarine, Mango, 

In [134]:
layout = dict(height=400, width=1200, title='class distribution in training data',
             legend=dict(orientation='h'), yaxis=dict(title='class count'))
fig = go.Figure(data=[trace1], layout=layout)
iplot(fig)

In [135]:
x1, y1 = create_stack_bar_data('classes', testing_data)
x1 = list(validation_generator.class_indices.keys())

trace1 = go.Bar(x=x1, y=y1, opacity=0.75, name="class count")
layout = dict(height=400, width=1100, title='class distribution in validation data', 
              legend=dict(orientation="h"), yaxis = dict(title = 'class count'))
fig = go.Figure(data=[trace1], layout=layout)
iplot(fig)

보다 싶이 모든 클래스가 균형적으로(잘 분포되어 있다는 뜻) 되어있음을 볼 수 있다. (validation set도)

## 4. Building and Compiling the Model

### 4.1. Building the Models
여기 ImageNet 가중치로 되어있는 ResNet-50을 로드한다. 가장 마지막 층을 제거하고 클래스의 개수만큼의 노드로 새로운 층을 추가한다. 그리고 모델 설계를 끝내기 위해 우리의 층(신경망)을 추가한다.

> 참고 : https://datascienceschool.net/view-notebook/958022040c544257aa7ba88643d6c032/

#### 4.1.1. Building Pretrained Model

In [136]:
# 미리 학습된 가중치로 inception을 임포트한다.
# 단 완전연결계층은 포함하지 않는다.
from tensorflow.keras.applications.resnet50 import ResNet50

inception_base = ResNet50(include_top=False, weights='imagenet')

# 마지막에 1000으로 분류하는 완전연결계층(Dense) 제거
# global spatial average pooling 계층 추가
x = inception_base.output
x = GlobalAveragePooling2D()(x)

# 완전 연결 계층 추가
x = Dense(512, activation='relu')(x)
# 131개의 과일을 분류하기 위한 완전연결계층 추가
predictions = Dense(65, activation='softmax')(x)

# 완성된 신경망을 생성
inception_transfer = Model(inputs=inception_base.input, outputs=predictions)

#### 4.1.2. Building the Vanilla Model

In [137]:
# 동일하지만 가중치 값이 없음
inception_base_vanilla = applications.ResNet50(weights=None, include_top=False)

# 아까와 동일하게 계층 추가
x = inception_base_vanilla.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(65, activation='softmax')(x)
inception_transfer_vanilla = Model(inputs=inception_base_vanilla.input, outputs=predictions)

### 4.2. Compiling the Models
비용함수(목적함수)와 최적화 알고리즘, 평가지표를 성정한다.

In [138]:
inception_transfer.compile(loss='categorical_crossentropy',
                          optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                          metrics=['accuracy'])
inception_transfer_vanilla.compile(loss='categorical_crossentropy',
                                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                                  metrics=['accuracy'])

In [139]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12544219946489836919
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7403868652066542137
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2376482744844670604
physical_device_desc: "device: XLA_GPU device"
]


## 5. Training and Validating the Pretrained Model
`fit_generator`를 사용할 것이다. 왜냐하면 데이터를 가져오는 ImageDataGenerator를 사용하기 때문.

In [140]:
import tensorflow as tf
with tf.device("/device:GPU:0"):
    history_pretrained = inception_transfer.fit_generator(
    train_generator,
    epochs=5, shuffle = True, verbose = 1, validation_data = validation_generator)

Epoch 1/5


InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[16,65] labels_size=[16,131]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits (defined at <ipython-input-140-a2a71863bdd6>:5) ]] [Op:__inference_train_function_64318]

Function call stack:
train_function


In [ ]:
with tf.device('/device:XLA_GPU:0'):
    history_vanilla = inception_transfer_vanilla.fit_generator(train_generator, epochs=5, shuffle=True,
                                                              verbose=1, validation_data=validation_generator)

In [ ]:
import matplotlib.pyplot as plt

# accuracy
plt.plot(history_pretrained.history['val_acc'])
plt.plot(history_vanilla.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanilla'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_pretrained.history['val_loss'])
plt.plot(history_vanilla.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Pretrained', 'Vanilla'], loc='upper left')
plt.show()